# Esercitazione Guidata: Analisi di Dati con Python e Pandas

Benvenuti a questa esercitazione pratica! L'obiettivo di oggi è imparare a esplorare, pulire e visualizzare un dataset reale utilizzando le librerie Python che sono il pane quotidiano di ogni Data Analyst: **Pandas**, **NumPy** e **Matplotlib**.

Lavoreremo con il dataset `Dataset Abitudini Sportive`, che raccoglie informazioni sulle abitudini di allenamento di diverse persone. Impareremo a scoprire cosa si nasconde dietro i numeri e a rispondere a domande concrete basate sui dati.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')

## 1. Caricamento e Prima Ispezione del Dataset

In [ ]:
file_path = 'Dataset Abitudini Sportive.csv'
df = pd.read_csv(file_path)
df_originale = df.copy()

### `df.head()` e `df.info()`
Diamo una prima occhiata ai dati per avere un'idea delle colonne, dei tipi di dato e dei valori mancanti.

In [ ]:
print("Prime 5 righe:")
display(df.head())
print("\nInformazioni sul DataFrame:")
df.info()

## 2. Analisi Esplorativa e Individuazione di Anomalie

In [ ]:
df.describe()

### Approfondimento: Identificare Outlier con l'IQR
Un valore `min` di `-999` è un chiaro errore. Un metodo statistico per trovare outlier è l'**Interquartile Range (IQR)**.

In [ ]:
Q1 = df['ore_settimanali_allenamento'].quantile(0.25)
Q3 = df['ore_settimanali_allenamento'].quantile(0.75)
IQR = Q3 - Q1
limite_inferiore = Q1 - 1.5 * IQR
limite_superiore = Q3 + 1.5 * IQR
print(f"Range di normalità per le ore di allenamento: [{limite_inferiore:.2f}, {limite_superiore:.2f}]")

**Esercizio A:** Calcola l'IQR e i limiti per `attrezzatura_comprata_annualmente`.

In [ ]:
# Scrivi qui il tuo codice

## 3. Pulizia dei Dati
Standardizziamo i valori anomali in `np.nan` e convertiamo le colonne in numeriche.

In [ ]:
df_pulito = df.copy()
df_pulito.replace([-999, '***', ''], np.nan, inplace=True)
for col in ['ore_settimanali_allenamento', 'attrezzatura_comprata_annualmente', 'soddisfazione_allenamento', 'Y']:
    df_pulito[col] = pd.to_numeric(df_pulito[col], errors='coerce')
df_pulito.isnull().sum()

### Strategie per Gestire i Dati Mancanti
**Strategia 1: Eliminazione (`dropna`)** - Rimuove le righe con NaN. Rischioso se si perdono troppi dati.

In [ ]:
print(f"Forma originale: {df_pulito.shape}")
print(f"Forma dopo dropna(): {df_pulito.dropna().shape}")

**Strategia 2: Imputazione (Media/Mediana/Moda)** - Sostituisce i NaN con un valore. Useremo questo approccio.

In [ ]:
for colonna in df_pulito.columns:
    if df_pulito[colonna].isnull().any():
        if pd.api.types.is_numeric_dtype(df_pulito[colonna]):
            df_pulito[colonna].fillna(df_pulito[colonna].median(), inplace=True)
        else:
            df_pulito[colonna].fillna(df_pulito[colonna].mode()[0], inplace=True)
df_pulito.isnull().sum()

## 4. Visualizzazione: L'Importanza della Pulizia

### Guida alla Scelta del Grafico
- **Istogramma**: Ottimo per vedere la **distribuzione** di una variabile numerica (es. `ore_settimanali_allenamento`). Mostra dove si concentrano i valori.
- **Grafico a Barre**: Perfetto per confrontare le **frequenze** di una variabile categorica (es. `obiettivo_allenamento_label`). Mostra quale categoria è la più comune.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
axes[0].hist(df_originale['ore_settimanali_allenamento'], bins=30, color='red')
axes[0].set_title('PRIMA della Pulizia')
axes[1].hist(df_pulito['ore_settimanali_allenamento'], bins=15, color='green')
axes[1].set_title('DOPO la Pulizia')
plt.show()

**Esercizio B:** Crea un confronto "Prima e Dopo" per `attrezzatura_comprata_annualmente`.

In [ ]:
# Scrivi qui il tuo codice

In [ ]:
df_pulito['obiettivo_allenamento_label'].value_counts().plot(kind='bar', figsize=(10,6), title='Frequenza Obiettivi di Allenamento')
plt.show()
df_pulito['tempo_riposo_settimanale_label'].value_counts().plot(kind='bar', figsize=(10,6), title='Frequenza Riposo Settimanale', color='skyblue')
plt.show()

## 5. Verifica delle Ipotesi

In [ ]:
print(df_pulito.groupby('obiettivo_allenamento_label')['attrezzatura_comprata_annualmente'].agg(['mean', 'median']))

**Interpretazione:** I dati supportano fortemente l'ipotesi. La spesa media per 'Miglioramento muscolare' è di 227.81 €, contro i 171.74 € per 'Perdita peso'. La differenza è ancora più marcata nella mediana (226.50 € vs 160.00 €), il che ci dice che la tendenza non è dovuta a pochi valori estremi. Questo suggerisce che raggiungere obiettivi di massa muscolare richiede mediamente un investimento economico superiore.

In [ ]:
df_pulito.groupby('frequenza_allenamento')['soddisfazione_allenamento'].mean().plot(kind='bar', figsize=(10,6), title='Soddisfazione Media per Frequenza di Allenamento')
plt.ylabel('Soddisfazione Media (1-10)')
plt.show()

**Interpretazione:** Il grafico mostra una relazione quasi lineare: all'aumentare dei giorni di allenamento, la soddisfazione media cresce costantemente. Si passa da una media di ~5.6 per chi si allena 1-2 volte a settimana a oltre 8 per chi si allena 6-7 volte. L'ipotesi è chiaramente confermata dai dati.

## Soluzioni

In [ ]:
Q1 = df['attrezzatura_comprata_annualmente'].quantile(0.25)
Q3 = df['attrezzatura_comprata_annualmente'].quantile(0.75)
IQR = Q3 - Q1
limite_inferiore = Q1 - 1.5 * IQR
limite_superiore = Q3 + 1.5 * IQR
print(f"Soluzione Esercizio A: Range di normalità: [{limite_inferiore:.2f}, {limite_superiore:.2f}]")

In [ ]:
print("Soluzione Esercizio B:")
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
axes[0].hist(df_originale['attrezzatura_comprata_annualmente'], bins=30, color='orange')
axes[0].set_title('PRIMA')
axes[1].hist(df_pulito['attrezzatura_comprata_annualmente'], bins=15, color='blue')
axes[1].set_title('DOPO')
plt.show()